In [5]:
# pip install openai

In [36]:
from openai import OpenAI
from keys import openai_key

# Function to use OpenAI autocomplete
def generate_autocomplete(question, temperature=0, top_p=1, n=1, system_prompt="You are a helpful assistant."):
    """
    Generates an autocomplete response using OpenAI's GPT model.
    
    Parameters:
        prompt_text (str): The initial text to complete.
        max_tokens (int): The Max number of tokens to generate in the completion.
        temperature (float): Controls the creativity (0.0 = deterministic, higher = more creative).
        top_p (float): Controls the probability threshold for vocabulary sampling.
        n (int): Number of completions to generate.
    
    Returns:
        str: The generated completion text.
    """
    # Make sure to set your OpenAI API key here
    client = OpenAI(
        # This is the default and can be omitted
        api_key=openai_key,
        )
    
    try:
        # Use the ChatCompletion API (chat-based interface)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
        )
        
        # Extract and return the generated text from the response
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Error: {e}"


# Example usage
question = "What is the capital of France?"
result = generate_autocomplete(question)
print(result)


The capital of France is Paris.


In [57]:
# task = """You are an intelligent SQL assistant. Your job is to generate SQL queries based on the provided database schema.

# Rules:
# 1. Always respond with the SQL query inside a JSON object using the format: `{"query": "---query---"}`.
# 2. Ensure the query is based strictly on the tables and data provided in the schema.
# 3. If a query requests information not available in the tables, return `{"query": ""}` (an empty query).
# 4. The SQL query must be syntactically correct and optimized for performance.
# 5. Avoid making assumptions about the data beyond what is provided in the schema.
# 6. Do not include any comments or explanations in the response—just the JSON object with the query.

# You are a precise SQL assistant that only returns SQL queries formatted in JSON.

# """

task = """You are an intelligent SQL assistant. Your job is to generate SQL queries based on the provided database schema.

Rules:
1. Always respond with the SQL query only, without any additional explanation.
2. Ensure the query is based strictly on the tables and data provided in the schema.
3. If a query requests information not available in the tables, return an empty response.
4. The SQL query must be syntactically correct and optimized for performance.
5. Avoid making assumptions about the data beyond what is provided in the schema.
6. Do not include any comments or explanations in the response—just the SQL query.

You are a precise SQL assistant that only returns valid SQL queries."""

# read schema from file tables.txt
with open("tables.txt", "r") as f:
    schema = f.read()

# read examples from file examples.txt
with open("examples.txt", "r") as f:
    examples = f.read()

# Combine the task description, schema, and examples
system_prompt = task + schema + examples

In [58]:
question = "get all running campaign?"
result = generate_autocomplete(question, system_prompt=system_prompt)
print(result)

```sql
SELECT 
    campaign.id AS campaign_id, 
    campaign.name AS campaign_name, 
    adset.start_date, 
    adset.end_date, 
    adset.status
FROM 
    sponsor_adset AS adset
INNER JOIN 
    sponsor_campaign AS campaign 
ON 
    adset.campaign_id = campaign.id
WHERE 
    adset.status = 'DELIVERING';
```
